<a href="https://colab.research.google.com/github/shukhany/perm-eb2-approval-ml/blob/main/Perm_Eb2_Approval_rate_Enhanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EB-2 PERM Approval Model (Enhanced with Candidate Education)
## Based on your existing workflow + education gap feature

**What's New:**
- Added candidate's actual education level
- Education gap calculation (overqualification detection)
- Improved model accuracy

**Everything else stays the same:**
- Same data loading process
- Same MLflow integration
- Same model selection logic

In [28]:
# ============================================================
# CELL 1: Imports + MLflow setup
# ============================================================

!pip install -q mlflow==2.16.0 openpyxl scikit-learn==1.5.1 joblib databricks-sdk

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score, accuracy_score, f1_score,
    precision_score, recall_score
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

import mlflow, mlflow.sklearn
from databricks.sdk import WorkspaceClient
import joblib
import os

print("✅ Imports done")

✅ Imports done


In [29]:
# ============================================================
# CELL 2: Configure MLflow (Databricks or local)
# ============================================================

def setup_mlflow_or_fallback(user_email: str, exp_suffix: str):
    host  = os.environ.get("DATABRICKS_HOST") or ""
    token = os.environ.get("DATABRICKS_TOKEN") or ""
    using_dbx = False
    if host and token.startswith("dapi") and len(token) >= 30:
        try:
            _ = WorkspaceClient(host=host.strip(), token=token.strip()).current_user.me()
            mlflow.set_tracking_uri("databricks")
            mlflow.set_experiment(f"/Users/{user_email}/{exp_suffix}")
            print("✅ Using Databricks MLflow:", mlflow.get_tracking_uri())
            using_dbx = True
        except Exception as e:
            print("⚠️ Databricks auth failed → falling back to local MLflow. Reason:", e)
    if not using_dbx:
        local_dir = "/content/mlruns"
        os.makedirs(local_dir, exist_ok=True)
        mlflow.set_tracking_uri(f"file:{local_dir}")
        mlflow.set_experiment(exp_suffix + "_LOCAL")
        print("✅ Using LOCAL MLflow:", mlflow.get_tracking_uri(), "| Experiment:", exp_suffix + "_LOCAL")
    return using_dbx

USER_EMAIL = "shukhany@asu.edu"
EXP_NAME   = "Perm_EB2_MultiYear_2022_2024"
_ = setup_mlflow_or_fallback(USER_EMAIL, EXP_NAME)

2025/12/01 06:34:51 INFO mlflow.tracking.fluent: Experiment with name '/Users/shukhany@asu.edu/Perm_EB2_MultiYear_2022_2024' does not exist. Creating a new experiment.


✅ Using Databricks MLflow: databricks


In [30]:
# ============================================================
# CELL 3: Upload + load PERM_2022/2023/2024.xlsx
# ============================================================

from google.colab import files

print("⬆️ Please upload: PERM_2022.xlsx, PERM_2023.xlsx, PERM_2024.xlsx")
uploaded = files.upload()

year_files = {}
for fname in uploaded.keys():

    if "2022" in fname:
        year_files[2022] = fname
    elif "2023" in fname:
        year_files[2023] = fname
    elif "2024" in fname:
        year_files[2024] = fname

print("Detected files by year:", year_files)

dfs = []
for year, fname in year_files.items():
    print(f"📥 Loading {fname} as year {year}")
    df = pd.read_excel(fname, engine="openpyxl")
    df["FISCAL_YEAR"] = year
    dfs.append(df)

perm_raw = pd.concat(dfs, ignore_index=True)
print("Combined shape (2022–2024):", perm_raw.shape)

⬆️ Please upload: PERM_2022.xlsx, PERM_2023.xlsx, PERM_2024.xlsx


Saving PERM_2024.xlsx to PERM_2024 (1).xlsx
Saving PERM_2023.xlsx to PERM_2023.xlsx
Saving PERM_2022.xlsx to PERM_2022.xlsx
Detected files by year: {2024: 'PERM_2024 (1).xlsx', 2023: 'PERM_2023.xlsx', 2022: 'PERM_2022.xlsx'}
📥 Loading PERM_2024 (1).xlsx as year 2024
📥 Loading PERM_2023.xlsx as year 2023
📥 Loading PERM_2022.xlsx as year 2022
Combined shape (2022–2024): (313285, 158)


In [31]:
# ============================================================
# CELL 4: Select core columns, build target
# ============================================================

required_cols = [
    "CASE_STATUS",
    "PW_SOC_CODE",
    "NAICS_CODE",
    "PW_WAGE",
    "PW_UNIT_OF_PAY",
    "WAGE_OFFER_FROM",
    "WAGE_OFFER_TO",
    "WAGE_OFFER_UNIT_OF_PAY",
    "MINIMUM_EDUCATION",
    "WORKSITE_STATE",
    "FW_OWNERSHIP_INTEREST",
    "FISCAL_YEAR",
]

missing = [c for c in required_cols if c not in perm_raw.columns]
if missing:
    raise ValueError(f"❌ Missing expected columns in PERM data: {missing}")

perm_model = perm_raw[required_cols].copy()

# Target: 1 = Certified, 0 = everything else
status = perm_raw["CASE_STATUS"].astype(str).str.upper()
perm_model["TARGET_APPROVED"] = status.eq("CERTIFIED").astype(int)

print("perm_model shape (before cleaning):", perm_model.shape)
print("Approval rate (raw):", perm_model["TARGET_APPROVED"].mean())

perm_model shape (before cleaning): (313285, 13)
Approval rate (raw): 0.4107505945065994


In [32]:
# ============================================================
# CELL 5: Wage normalization + cleaning
# ============================================================

def to_annual(amount, unit_col):
    """
    Convert wages with different units to annualized numbers.
    """
    unit = unit_col.astype(str).str.upper().str.strip()
    amt = pd.to_numeric(amount, errors="coerce")

    factors = (
        unit.replace({
            "HOUR": 2080,
            "HR": 2080,
            "WEEK": 52,
            "WK": 52,
            "BI-WEEKLY": 26,
            "BIWEEKLY": 26,
            "MONTH": 12,
            "MO": 12,
            "YEAR": 1,
            "YR": 1,
            "": np.nan
        })
    )

    return amt * pd.to_numeric(factors, errors="coerce")

print("🧹 Cleaning data...\n")
print(f"Starting shape: {perm_model.shape}")

# Drop rows missing critical fields
critical = [
    "PW_WAGE", "PW_UNIT_OF_PAY",
    "WAGE_OFFER_FROM", "WAGE_OFFER_TO", "WAGE_OFFER_UNIT_OF_PAY",
    "PW_SOC_CODE", "NAICS_CODE",
    "MINIMUM_EDUCATION", "WORKSITE_STATE", "FW_OWNERSHIP_INTEREST"
]
before_crit = len(perm_model)
perm_model = perm_model.dropna(subset=critical)
print(f"After dropping missing critical fields: {len(perm_model):,} rows (-{before_crit - len(perm_model):,})")

# Replace inf
perm_model = perm_model.replace([np.inf, -np.inf], np.nan)

# Annualize prevailing wage & offered wage
perm_model["PW_WAGE_ANNUAL"] = to_annual(perm_model["PW_WAGE"], perm_model["PW_UNIT_OF_PAY"])

offer_mid = perm_model[["WAGE_OFFER_FROM", "WAGE_OFFER_TO"]].mean(axis=1)
perm_model["OFFER_WAGE_ANNUAL"] = to_annual(offer_mid, perm_model["WAGE_OFFER_UNIT_OF_PAY"])

# Wage ratio
perm_model["WAGE_RATIO"] = perm_model["OFFER_WAGE_ANNUAL"] / perm_model["PW_WAGE_ANNUAL"]

# Drop rows where we couldn't compute wages
before_wage = len(perm_model)
perm_model = perm_model.dropna(subset=["PW_WAGE_ANNUAL", "OFFER_WAGE_ANNUAL", "WAGE_RATIO"])
print(f"After wage computation: {len(perm_model):,} rows (-{before_wage - len(perm_model):,})")

# Remove insane wage ratios
before_outlier = len(perm_model)
perm_model = perm_model[(perm_model["WAGE_RATIO"] >= 0.5) & (perm_model["WAGE_RATIO"] <= 5.0)]
print(f"Removed {before_outlier - len(perm_model):,} wage ratio outliers")

print("\n✅ Cleaning complete")
print("Final shape:", perm_model.shape)
print("Approval rate:", perm_model["TARGET_APPROVED"].mean())

🧹 Cleaning data...

Starting shape: (313285, 13)
After dropping missing critical fields: 112,384 rows (-200,901)


/tmp/ipython-input-1826290688.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  unit.replace({


After wage computation: 112,384 rows (-0)
Removed 70 wage ratio outliers

✅ Cleaning complete
Final shape: (112314, 16)
Approval rate: 0.424773403137632


/tmp/ipython-input-1826290688.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  unit.replace({


In [36]:
# ============================================================
# CELL 6: Prepare features + train/test split
# ============================================================

feature_cols = [
    "PW_SOC_CODE",
    "NAICS_CODE",
    "PW_UNIT_OF_PAY",
    "WAGE_OFFER_UNIT_OF_PAY",
    "MINIMUM_EDUCATION",
    "WORKSITE_STATE",
    "FW_OWNERSHIP_INTEREST",
    "PW_WAGE_ANNUAL",
    "OFFER_WAGE_ANNUAL",
    "WAGE_RATIO",
]

numeric_features = ["PW_WAGE_ANNUAL", "OFFER_WAGE_ANNUAL", "WAGE_RATIO"]
categorical_features = [c for c in feature_cols if c not in numeric_features]


# ============================================================
# 🔧 FIX MIXED-TYPE CATEGORICAL COLUMNS HERE
# ============================================================

for col in categorical_features:
    perm_model[col] = perm_model[col].astype("string").fillna("UNKNOWN")

# ============================================================
# Continue as normal
# ============================================================

X = perm_model[feature_cols].copy()
y = perm_model["TARGET_APPROVED"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train shape:", X_train.shape, " Test shape:", X_test.shape)
print("Overall approval (y mean):", y.mean())

Train shape: (89851, 10)  Test shape: (22463, 10)
Overall approval (y mean): 0.424773403137632


In [37]:
# ============================================================
# CELL 7: Preprocessing pipeline + models
# ============================================================

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

models = {
    "LogisticRegression": LogisticRegression(
        max_iter=1000,
        n_jobs=-1,
        class_weight="balanced"
    ),
    "DecisionTree": DecisionTreeClassifier(
        max_depth=10,
        min_samples_leaf=100,
        random_state=42
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        max_depth=12,
        min_samples_leaf=100,
        n_jobs=-1,
        random_state=42
    ),
    "GaussianNB": GaussianNB(),
    "NeuralNet_MLP": MLPClassifier(
        hidden_layer_sizes=(32, 16),
        activation="relu",
        alpha=1e-3,
        learning_rate_init=1e-3,
        max_iter=50,
        random_state=42
    ),
}

print("Models defined:", list(models.keys()))

Models defined: ['LogisticRegression', 'DecisionTree', 'RandomForest', 'GaussianNB', 'NeuralNet_MLP']


In [40]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.naive_bayes import GaussianNB

def to_dense(X):
    """
    Convert sparse matrix to dense numpy array.
    Used only for GaussianNB, which doesn't accept sparse input.
    """
    if hasattr(X, "toarray"):
        return X.toarray()
    return X

In [41]:
# ============================================================
# CELL 8: Train models, compute metrics, log to MLflow
#        (with GaussianNB dense conversion, pickle-safe)
# ============================================================

results = []

for name, base_model in models.items():
    print(f"\n=== Training {name} ===")

    # GaussianNB needs dense input
    needs_dense = isinstance(base_model, GaussianNB)

    steps = [("preprocess", preprocess)]

    if needs_dense:
        # Use named function (no lambda!) so joblib can pickle the pipeline
        steps.append(
            (
                "to_dense",
                FunctionTransformer(
                    to_dense,
                    accept_sparse=True,
                ),
            )
        )

    steps.append(("clf", base_model))
    clf = Pipeline(steps=steps)

    with mlflow.start_run(run_name=name):
        # ---- Fit ----
        clf.fit(X_train, y_train)

        # ---- Probabilities for positive class ----
        if hasattr(clf["clf"], "predict_proba"):
            y_proba = clf.predict_proba(X_test)[:, 1]
        else:
            scores = clf.decision_function(X_test)
            smin, smax = scores.min(), scores.max()
            y_proba = (scores - smin) / (smax - smin + 1e-9)

        y_pred = (y_proba >= 0.5).astype(int)

        # ---- Metrics ----
        test_accuracy  = accuracy_score(y_test, y_pred)
        test_f1        = f1_score(y_test, y_pred, zero_division=0)
        test_precision = precision_score(y_test, y_pred, zero_division=0)
        test_recall    = recall_score(y_test, y_pred, zero_division=0)
        roc_auc        = roc_auc_score(y_test, y_proba)

        # ---- Log to MLflow ----
        mlflow.log_metric("test_accuracy",  test_accuracy)
        mlflow.log_metric("test_f1",        test_f1)
        mlflow.log_metric("test_precision", test_precision)
        mlflow.log_metric("test_recall",    test_recall)
        mlflow.log_metric("roc_auc",        roc_auc)

        print(
            f"{name} | AUC: {roc_auc:.4f} | Acc: {test_accuracy:.4f} | "
            f"F1: {test_f1:.4f} | Prec: {test_precision:.4f} | Rec: {test_recall:.4f}"
        )

        # ---- Save fitted pipeline ----
        joblib.dump(clf, f"model_{name}.pkl")

        # ---- Store in results table ----
        results.append({
            "model": name,
            "roc_auc": roc_auc,
            "accuracy": test_accuracy,
            "f1": test_f1,
            "precision": test_precision,
            "recall": test_recall,
        })

results_df = pd.DataFrame(results).sort_values("roc_auc", ascending=False)
results_df


=== Training LogisticRegression ===
LogisticRegression | AUC: 0.5954 | Acc: 0.5663 | F1: 0.5230 | Prec: 0.4908 | Rec: 0.5596


2025/12/01 07:11:37 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression at: https://dbc-ca0a3ce7-da46.cloud.databricks.com/ml/experiments/2146850892363742/runs/6e3925237ac54fa2a967269ec5ffb78e.
2025/12/01 07:11:37 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dbc-ca0a3ce7-da46.cloud.databricks.com/ml/experiments/2146850892363742.



=== Training DecisionTree ===
DecisionTree | AUC: 0.6046 | Acc: 0.6027 | F1: 0.3604 | Prec: 0.5699 | Rec: 0.2636


2025/12/01 07:11:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run DecisionTree at: https://dbc-ca0a3ce7-da46.cloud.databricks.com/ml/experiments/2146850892363742/runs/e08c2e1f3d804d6d9019392a16a8ffa0.
2025/12/01 07:11:43 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dbc-ca0a3ce7-da46.cloud.databricks.com/ml/experiments/2146850892363742.



=== Training RandomForest ===
RandomForest | AUC: 0.5938 | Acc: 0.5752 | F1: 0.0000 | Prec: 0.0000 | Rec: 0.0000


...


=== Training NeuralNet_MLP ===


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


NeuralNet_MLP | AUC: 0.6561 | Acc: 0.6320 | F1: 0.5360 | Prec: 0.5770 | Rec: 0.5004


2025/12/01 07:13:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run NeuralNet_MLP at: https://dbc-ca0a3ce7-da46.cloud.databricks.com/ml/experiments/2146850892363742/runs/a68643aed51c4ceaad6c152d93eb090c.
2025/12/01 07:13:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dbc-ca0a3ce7-da46.cloud.databricks.com/ml/experiments/2146850892363742.


,model,roc_auc,accuracy,f1,precision,recall
4,NeuralNet_MLP,0.656079,0.631973,0.536005,0.577039,0.500419
1,DecisionTree,0.604603,0.602680,0.360444,0.569907,0.263572
0,LogisticRegression,0.595360,0.566309,0.522965,0.490809,0.559631
2,RandomForest,0.593781,0.575213,0.000000,0.000000,0.000000
3,GaussianNB,0.508789,0.439167,0.595414,0.429246,0.971494


In [42]:
import joblib
import os

best_model_path = "model_NeuralNet_MLP.pkl"   # winner

print("Looking for:", best_model_path, "| Exists?", os.path.exists(best_model_path))

best_model = joblib.load(best_model_path)

best_deploy_path = "model_perm_best.pkl"
joblib.dump(best_model, best_deploy_path)

print("✅ Saved deployment model as:", best_deploy_path)
print("Exists now?", os.path.exists(best_deploy_path))

from google.colab import files
files.download(best_deploy_path)

Looking for: model_NeuralNet_MLP.pkl | Exists? True
✅ Saved deployment model as: model_perm_best.pkl
Exists now? True


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>